# RISC-V Qemu

Since there are no available RISCV64 CPUs just yet, we need to use Qemu with the latest RISC-V patches to run our distributions.

Following this Notebook you can have a development or testing machine ready to use in very little time at all.

Note: This guide is still a work in progress.
TODO:
- Add comments about what the commands do
- Add info about how to create slack.img

https://github.com/riscv/riscv-qemu
https://rwmj.wordpress.com/2018/02/15/fedora-risc-v-runnable-stage-4-disk-images/

In [ ]:
cd ~
sudo apt-get install -y git flex bison lftp
sudo apt-get build-dep -y qemu

In [ ]:
git clone https://github.com/riscv/riscv-qemu.git

In [ ]:
cd riscv-qemu

In [ ]:
mkdir build
cd build
../configure

In [ ]:
make -j6

In [ ]:
sudo make install

In [ ]:
cd ~
mkdir fedora-stage4
cd fedora-stage4
lftp -e "mirror -c;exit" https://fedorapeople.org/groups/risc-v/disk-images/

In [ ]:
xz -dk stage4-disk.img.xz

**From a terminal**:
```
sudo qemu-system-riscv64 \
  -nographic -machine virt -m 6G -smp 6 \
  -kernel bbl \
  -append "console=ttyS0 ro root=/dev/vda init=/init" \
  -device virtio-blk-device,drive=hd0 \
  -drive file=stage4-disk.img,format=raw,id=hd0 \
  -device virtio-net-device,netdev=usernet \
  -netdev tap,id=usernet \
  -device virtio-blk-device,drive=hd1
  -drive file=slack.img,format=raw,id=hd1
```

